<a href="https://colab.research.google.com/github/RoshanPShetty/Campus-Design-And-Copy/blob/master/Copy_of_Homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import random
import sys
import pandas as pd
import altair as alt

# The BufferManager class

In [119]:
#  This is a class definition for a buffer manager simulator
#  It is instantiated with a size (bufferPoolSize) and a string describing an implemented policy (policyName)
#  Calling method 'setWorkload' will establish a workload
#  Calling method 'simulate' will run the workload under the chosen policy
#  Calling method 'missRate' will return the percentage of page accesses which were cache misses.

class BufferManager:

    def __init__(self, bufferPoolSize, policyName):
        self.size = bufferPoolSize
        self.policyName = policyName
        self.buffer = [None]*self.size   # buffer[i] stores the pageID of frame i; empty frames contain None.
        self.used = [0]*self.size        # used[i] contains the clock value of the last time the page in frame i was used.
        self.timeIn = [0]*self.size
        self.cacheHit = 0
        self.cacheMiss = 0
        self.clock = 0
        self.workload = [ ]
        self.policyList = {              # This dictionary associates policy names with policy functions
            'RANDOM': self.policyRANDOM, 
            'LRU': self.policyLRU,
            'MRU': self.policyMRU,
            'MYSTERY': self.policyMYSTERY,
            'FIFO': self.policyFIFO
            }
        self.replacementPolicy = self.policyList[self.policyName]  # this chooses the desired policy function and assigns it to 'replacementPolicy' which holds a function object 
        random.seed('blah')     # set seed so results for random policy are repeatable
  
    def missRate(self):     # return the cache miss rate after a simulation 
        return float(self.cacheMiss) / (self.cacheHit + self.cacheMiss)
  
    def setWorkload(self, workload):
        self.workload = workload[:]
  
    def simulate(self):
        while len(self.workload) > 0:
            page = self.workload.pop(0)
            self.request(page)
  
    def request(self, pageID):    # this is the main logic in which the buffer manager responds to a request for 'pageID'
        if pageID in self.buffer:   # page is already in buffer
            self.cacheHit += 1
            # record that the page was accessed in the 'used' dictionary
            frameUsed = self.buffer.index(pageID)   # List.index(element) returns the first position in the list where 'element' occurs
        else:             # page is not present in buffer
            self.cacheMiss += 1
            if None in self.buffer:     # there is free space
                frameUsed = self.buffer.index(None)   # find first available frame
                self.buffer[ frameUsed ] = pageID     # place pageID in the frame
                self.timeIn[ frameUsed ] = self.clock   # maintain timeIn for FIFO
            else:           # there is no free space -- something must be replaced
                frameUsed = self.replacementPolicy()  # use the policy to find the frame to replace
                self.buffer[ frameUsed ] = pageID
                self.timeIn[ frameUsed ] = self.clock # maintain timeIn for FIFO
        self.used[ frameUsed ] = self.clock # no matter which case happened, record that page in the touched frame was accessed
        self.clock += 1           # move clock forward

        #################################################################################################
        #  Replacement Policies
        
    def policyLRU(self):
        # implement this
        frame = self.used.index( min(self.used) )
        return frame 

    def policyMRU(self):
        frame = self.used.index( max(self.used) )
        return frame

    def policyRANDOM(self):     # this policy chooses a random frame to be replaced
        return random.randint(0,self.size-1)

    def policyMYSTERY(self):    # mystery policy.  What is this doing?
        temp = []
        for page in self.buffer:
            if page in self.workload:
                temp.append( self.workload.index(page) )
            else:
                temp.append( sys.maxsize )  # sys.maxsize is an integer larger than any practical list or string index
        maxIndex = max( temp )
        return temp.index( maxIndex )   # this is the frame choosen for replacement

    def policyFIFO(self):   
        # implement this
        frame = self.timeIn.index( min(self.timeIn) )
        return frame

# Workload generators
for a random sequence of page accesses and a sequence arises from a nested-loops implementation.

In [120]:
#  Workload generators

#  This function generates a workload of page accesses: that is, a sequence of pageIDs.
#  This workload is a random sequence of pageIDs of the specified length, workloadLength.
def randomWorkload( dataSize, workloadLength, seed ):
    random.seed(seed)
    workload = []
    for i in range(workloadLength):
        workload.append( random.randint(0,dataSize-1) )
    return workload

#  This function generates a workload of page accesses: that is, a sequence of pageIDs.
#  This workload is the sequence of pageIDs that would result from running a nested loops computation of a join or crossproduct 
#  over relations R and S where each relation has size m = (dataSize/2)
#  The length of the workload is 2(m^2)
def nestedLoopsWorkload( dataSize ):
    workload = []
    tableR = range(0,dataSize//2)
    tableS = range(dataSize//2,dataSize)
    for i in tableR:
        for j in tableS:
            workload.append(i)
            workload.append(j)
    return workload


In [121]:
def run_policies(workload, bufferSize=100):

    results = []
    # Run some buffer manager simulations, for different buffer sizes
    for policy in ['RANDOM','LRU','MRU','MYSTERY','FIFO']:        # Add new policies here, as needed
        for bufferSize in [5, 10, 25, 50, 75, 90, 100]:   # These buffer sizes don't need to be changed
            B = BufferManager( bufferSize, policy )
            B.setWorkload( workload )
            B.simulate()
            results.append([policy, B.size, B.missRate()])
    
    # return results as a Pandas dataframe
    return pd.DataFrame(results, columns = ['Policy','Buffer_Size' , 'Miss_Rate'])
  

# Simulate the random workload

In [122]:
# This is the number of data pages stored on disk.
# We assume stored pages have pageIDs from 0 to (dataSize-1)
dataSize = 100    # Leave this as-is; you don't have to change this.

In [123]:
rand_workload = randomWorkload( dataSize, 5000, seed='iLoveDatabases')
df = run_policies(rand_workload)
print(df)

# display chart
alt.Chart(df).mark_line(point=True).encode(x='Buffer_Size:Q', y='Miss_Rate:Q', color='Policy')

     Policy  Buffer_Size  Miss_Rate
0    RANDOM            5     0.9496
1    RANDOM           10     0.9028
2    RANDOM           25     0.7438
3    RANDOM           50     0.5050
4    RANDOM           75     0.2606
5    RANDOM           90     0.1142
6    RANDOM          100     0.0200
7       LRU            5     0.9560
8       LRU           10     0.9034
9       LRU           25     0.7544
10      LRU           50     0.5070
11      LRU           75     0.2620
12      LRU           90     0.1134
13      LRU          100     0.0200
14      MRU            5     0.9470
15      MRU           10     0.8972
16      MRU           25     0.7484
17      MRU           50     0.4966
18      MRU           75     0.2578
19      MRU           90     0.1108
20      MRU          100     0.0200
21  MYSTERY            5     0.7806
22  MYSTERY           10     0.6514
23  MYSTERY           25     0.4272
24  MYSTERY           50     0.2174
25  MYSTERY           75     0.0922
26  MYSTERY           90    

alt.Chart(...)

# Simulate the nested-loops workload

In [124]:
import numpy as np
nested_workload = nestedLoopsWorkload( dataSize )
df = run_policies(nested_workload)
print(df)

# display chart
alt.Chart(df).mark_line(point=True).encode(x='Buffer_Size:Q', y='Miss_Rate:Q', color='Policy')

     Policy  Buffer_Size  Miss_Rate
0    RANDOM            5     0.6066
1    RANDOM           10     0.5584
2    RANDOM           25     0.4412
3    RANDOM           50     0.1140
4    RANDOM           75     0.0348
5    RANDOM           90     0.0212
6    RANDOM          100     0.0200
7       LRU            5     0.5100
8       LRU           10     0.5100
9       LRU           25     0.5100
10      LRU           50     0.5100
11      LRU           75     0.0200
12      LRU           90     0.0200
13      LRU          100     0.0200
14      MRU            5     0.9208
15      MRU           10     0.8218
16      MRU           25     0.5248
17      MRU           50     0.0298
18      MRU           75     0.0250
19      MRU           90     0.0220
20      MRU          100     0.0200
21  MYSTERY            5     0.4798
22  MYSTERY           10     0.4298
23  MYSTERY           25     0.2798
24  MYSTERY           50     0.0298
25  MYSTERY           75     0.0200
26  MYSTERY           90    

alt.Chart(...)

# 1(c) 
Why does the cache miss rate remain above zero, even when the buffer is as large as the
total number of data pages (i.e. equal to the value of dataSize)?

When a page is requested, we need to check if it is in the buffer pool? If so then it is cache hit, and we increment pin count. If not then it is cache miss. Since the first one is not present in the buffer pool, it will always be a cache miss. Therefore, the miss rate would never be zero regardless of the buffer size. 


# 1(d)
How does policyMYSTERY compute the page to replace and how does it perform in
comparison to the other policies? What challenge would you face in implementing policyMYSTERY
in a real system?

policyMYSTERY computes the page to replace by returning the index of a page with the max value from the buffer. When there are pages from the buffer that isn't in the workload, temp is appended to sys.maxsize and so would return the page the first index that isn't in the workload. If all of them are present in the workload, then index of all the pages are appended and the max is returned. As noticed in the graphs, policyMYSTERY's performance is the best in comparison to the rest of the policies. This is true in case of both the random workload and a nested-loops workload. We need to make sure that we take of sequential flooding when it comes to implementing policyMYSTERY. If the number of buffer frames is less than the number of pages in the file, then that would lead to difficulties. So this needs to be taken care of as. 




# 1(e)
For the nested loops workload, how would you explain the performance of the MRU
policy when the buffer size is greater than or equal to 50?

`nested_workload` calls the `nestedLoopsWorkload` would where we append `j` from `datasize//2` to `datasize`  for each `i` from `0` to `datasize//2` and so would result in the following structure:

`[0, 50, 0, 51, 0, 52, 0, 53, 0, 54, 0, 55, 0, 56, 0, 57, 0, 58, 0, 59, 0, 60, 0, 61, 0, 62, 0, 63, 0, 64, 0, 65, 0, 66, 0, 67, 0, 68, 0, 69, 0, 70, 0, 71, 0, 72, 0, 73, 0, 74, 0, 75, 0, 76, 0, 77, 0, 78, 0, 79, 0, 80, 0, 81, 0, 82, 0, 83, 0, 84, 0, 85, 0, 86, 0, 87, 0, 88, 0, 89, 0, 90, 0, 91, 0, 92, 0, 93, 0, 94, 0, 95, 0, 96, 0, 97, 0, 98, 0, 99, 1, 50, ... 49, 99]`

This implies that the `buffer size` is smaller than the number of pages, and so it would work much better with `MRU` than other policies, althought it's not true for all cases. In case of when the `buffer size` is 50 or greater, it would be better as it prevents sequential flooding. In  only 50 of them can be added at once. With MRU, the most recently used page would be replaced for each iteration. Therefore, we can see that the `miss_rate` is halved when we reach during the first iteration and for the rest of the iterations it is significantly reduced as only `1,2,...49` are missed and the rest of the elements were already hit. Therefore, when the `buffer size` is greater or equal to 50, the performance is very good as a result of the prevention of sequential flooding. 